In [29]:
import pandas as pd
import numpy as np
import random

from google.colab import drive
drive.mount('/content/drive')

np.random.seed(1)
random.seed(1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
TRAININFO = "/content/drive/My Drive/data/hit_song/train_info.tsv"
TRAINRANK =  "/content/drive/My Drive/data/hit_song/train_rank.csv"
TESTINFO = "/content/drive/My Drive/data/hit_song/test_info.tsv"
Track_info = "/content/drive/My Drive/data/hit_song/all_track_info.csv"
Audio_info = "/content/drive/My Drive/data/hit_song/all_track_audio_features.csv"

In [31]:
df_i = pd.read_csv(TRAININFO, delimiter='\t',encoding='utf-8')
df_r = pd.read_csv(TRAINRANK)
df_i_train = df_i.merge(df_r, left_on='ID', right_on='ID')
df_i_train["dataset"] = "train"

df_i_test = pd.read_csv(TESTINFO, delimiter='\t',encoding='utf-8')
df_i_test["label"] = np.nan
df_i_test["dataset"] = "test"

df = pd.concat([df_i_train, df_i_test])
df_track_info = pd.read_csv(Track_info)
df = df.merge(df_track_info, left_on='ID', right_on='ID')
df_audio_features = pd.read_csv(Audio_info)
df =df.merge(df_audio_features,left_on="ID",right_on="ID", how="left")
df = df[['ID','title','artist_name','lyric','label','dataset']]
df.head()

,ID,title,artist_name,lyric,label,dataset
0,1073748245,Đêm Chôn Dầu Vượt Biển,Như Quỳnh,Đêm nay anh gánh dầu ra biển anh chôn \r\nAnh ...,7.0,train
1,1073751978,Mùa Thu Trong Mưa,Minh Tuyết,Chiều mưa không có em\r\nbờ đá công viên âm th...,3.0,train
2,1073835561,Rồi Ánh Trăng Tan,Lưu Bích,Rồi ánh trăng cũng đang tan dần\r\nRồi ước mơ ...,6.0,train
3,1073856553,Còn Thương Rau Đắng Mọc Sau Hè,Như Quỳnh,Nắng hạ đi Mây trôi lang thang cho hạ buồn Coi...,2.0,train
4,1073929630,Người Điên Biết Yêu,Như Loan,"Ai trong tình yêu, ai không mơ mộng?\n\nNgu ng...",7.0,train


In [32]:
import re

df.drop([6707,6718,7426], inplace=True)
df.lyric.fillna("", inplace=True)
def cleanString(string):
  return re.sub('\W+',' ', string )
df.lyric = df.lyric.apply(lambda x: cleanString(x))
df.lyric = df.lyric.apply(lambda x: x.lower())
df.head()

,ID,title,artist_name,lyric,label,dataset
0,1073748245,Đêm Chôn Dầu Vượt Biển,Như Quỳnh,đêm nay anh gánh dầu ra biển anh chôn anh chôn...,7.0,train
1,1073751978,Mùa Thu Trong Mưa,Minh Tuyết,chiều mưa không có em bờ đá công viên âm thầm ...,3.0,train
2,1073835561,Rồi Ánh Trăng Tan,Lưu Bích,rồi ánh trăng cũng đang tan dần rồi ước mơ cũn...,6.0,train
3,1073856553,Còn Thương Rau Đắng Mọc Sau Hè,Như Quỳnh,nắng hạ đi mây trôi lang thang cho hạ buồn coi...,2.0,train
4,1073929630,Người Điên Biết Yêu,Như Loan,ai trong tình yêu ai không mơ mộng ngu ngơ từn...,7.0,train


In [33]:
def artist_func(string):
  return ''.join(string.split(' '))+ "_"
df['artist_name'] = df['artist_name'].apply(lambda x: artist_func(x))
df['lyric'] = df['artist_name'] + df['lyric']
df.head()

,ID,title,artist_name,lyric,label,dataset
0,1073748245,Đêm Chôn Dầu Vượt Biển,NhưQuỳnh_,NhưQuỳnh_đêm nay anh gánh dầu ra biển anh chôn...,7.0,train
1,1073751978,Mùa Thu Trong Mưa,MinhTuyết_,MinhTuyết_chiều mưa không có em bờ đá công viê...,3.0,train
2,1073835561,Rồi Ánh Trăng Tan,LưuBích_,LưuBích_rồi ánh trăng cũng đang tan dần rồi ướ...,6.0,train
3,1073856553,Còn Thương Rau Đắng Mọc Sau Hè,NhưQuỳnh_,NhưQuỳnh_nắng hạ đi mây trôi lang thang cho hạ...,2.0,train
4,1073929630,Người Điên Biết Yêu,NhưLoan_,NhưLoan_ai trong tình yêu ai không mơ mộng ngu...,7.0,train


In [34]:
def personalized(string):
  lst = string.split('_')[1].split(" ")
  for i in range(0,len(lst)):
    lst[i] = string.split("_")[0]+"_"+lst[i]
  return " ".join(lst)
df['lyric'] = df['lyric'].apply(lambda x: personalized(x))
df.head()

,ID,title,artist_name,lyric,label,dataset
0,1073748245,Đêm Chôn Dầu Vượt Biển,NhưQuỳnh_,NhưQuỳnh_đêm NhưQuỳnh_nay NhưQuỳnh_anh NhưQuỳn...,7.0,train
1,1073751978,Mùa Thu Trong Mưa,MinhTuyết_,MinhTuyết_chiều MinhTuyết_mưa MinhTuyết_không ...,3.0,train
2,1073835561,Rồi Ánh Trăng Tan,LưuBích_,LưuBích_rồi LưuBích_ánh LưuBích_trăng LưuBích_...,6.0,train
3,1073856553,Còn Thương Rau Đắng Mọc Sau Hè,NhưQuỳnh_,NhưQuỳnh_nắng NhưQuỳnh_hạ NhưQuỳnh_đi NhưQuỳnh...,2.0,train
4,1073929630,Người Điên Biết Yêu,NhưLoan_,NhưLoan_ai NhưLoan_trong NhưLoan_tình NhưLoan_...,7.0,train


In [35]:
df_train=df[df.dataset=='train']
print(df_train.shape)
df_train.drop_duplicates(subset='lyric', keep='first',inplace=True)
print(df_train.shape)

(9075, 6)
(4599, 6)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
from sklearn.feature_extraction.text import HashingVectorizer
vectorizer = HashingVectorizer(n_features=2**19)


In [0]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=99999)
oof = np.zeros(len(df_train))
labels= df_train.label

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train.label.values)):
  print("Fold {}".format(fold_))
  train_x, train_y = df_train['lyric'].iloc[trn_idx], df_train['label'].iloc[trn_idx]
  valid_x, valid_y = df_train['lyric'].iloc[val_idx], df_train['label'].iloc[val_idx]
  train_x = vectorizer.fit_transform(train_x).todense()
  valid_x = vectorizer.fit_transform(valid_x).todense()
  # clf = GaussianNB()
  clf = MultinomialNB()
  clf.fit(train_x, train_y)
  oof[val_idx] = clf.predict(valid_x)
  print("rmse_train: "+str(np.sqrt(mse(clf.predict(train_x), train_y))))
  print("rmse_val: "+str(np.sqrt(mse(oof[val_idx], valid_y))))

Fold 0


In [0]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer()
# vectors = vectorizer.fit_transform(df_train.lyric)
# feature_names = vectorizer.get_feature_names()
# dense = vectors.todense()
# denselist = dense.tolist()
# df_train_new = pd.DataFrame(denselist, columns=feature_names)
# df_train_new.head()